# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Guillem Barta Gonzàlez<br>
https://github.com/Willy8m/03_Algoritmos/SEMINARIO

Problema:
> 1. Sesiones de doblaje <br>

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. 

Los datos son: 
- Número de actores: 10 
- Número de tomas : 30
- Actores/Tomas : https://bit.ly/36D8IuK
  - 1 indica que el actor participa en la toma
  - 0 en caso contrario

### Información clave

Hay que encontrar que combinación de tomas resulta en el menor número de dias de doblaje.

Restricciones a tener en cuenta:
- Cada actor puede completar cómo máximo, 6 tomas al día.
- Los actores de una misma toma deben asistir el mismo día para grabar dicha toma.

(*) La respuesta es obligatoria

In [1]:
import math
import pandas as pd
import numpy as np
from decimal import Decimal

In [2]:
# Load data
df = pd.read_excel('Datos problema doblaje(30 tomas, 10 actores).xlsx', header = 1, index_col = 0)

In [3]:
# Drop columns and rows to keep only the data
data = df[:-2].copy() # drop two last rows
data.drop(columns=["Unnamed: 11", "Total"], inplace=True)  # drop two last columns
data = data.astype(int)

# Restrictions
max_shots = 6
data.sample(3)

1  2  3  4  5  6  7  8  9  10
Toma                               
3     0  1  0  0  1  0  1  0  0   0
22    1  1  1  1  0  0  0  0  0   0
20    1  0  1  1  1  0  0  0  0   0

__(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?__<br>

- Primera aproximación (respuesta mala):

Tenemos que calcular de cuántas maneras diferentes podemos ordenar las tomas, es decir las filas de nuestro dataset. Esta cantidad se obtiene con el factorial del número de elementos. *Nota: No hay que comprobar que haya filas duplicadas, ya que las tomas de por sí son distintas (aunque a efectos del problema si serian idénticas)*

In [4]:
# RESPUESTA MALA, sin considerar agrupaciones por día
print("Possibilities (wrong): ", '%.2E' % Decimal(math.factorial(30)))

Possibilities (wrong):  2.65E+32


- Análisis posterior, con referencia al estudio de Alberto Caldas Lima [1]:

Debido a que ahora hemos encontrado una manera de representar el espacio de soluciones, nos damos cuenta de que el número de posibilidades no se rige por la combinación de tomas, sinó por la agrupación, sin orden, en grupos de tomas por día. Así, las posibilidades van desde tener todas las tomas el mismo día, a tener todas las tomas separadas en días distintos (caso menos eficiente).

Para calcular el número de particiones posibles con 30 elementos usamos el número de Bell $B_{n}$ [2], donde "$n$" corresponde al número de elementos. De un algoritmo para calcular números de Bell de Rajeev Agrawal [3] se obtiene que:

In [5]:
# code credit: Rajeev Agrawal
# python program to find number of ways of partitioning it.
n = 30
s = [[0 for _ in range(n+1)] for _ in range(n+1)]
for i in range(n+1):
    for j in range(n+1):
        if j > i:
            continue
        elif(i==j):
            s[i][j] = 1
        elif(i==0 or j==0):
            s[i][j]=0
        else:
            s[i][j] = j*s[i-1][j] + s[i-1][j-1]
ans = 0
for i in range(0,n+1):
    ans+=s[n][i]
print("Possibilities: ", '%.2E' % Decimal(ans))

Possibilities:  8.47E+23


__¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?__

Teniendo en cuenta las restricciones, el número de posibbles combinaciones de tomas y días dependerá de la ocupación de los actores en cada toma. De módo que el número de posibles agrupaciones depende de los datos específicos de cada problema.

Es posible que exista una estructura de datos más adecuada que nos permita calcular este número.

__Modelo para el espacio de soluciones<br>__
__(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)__

- Primera aproximación (mala respuesta):

La estructura de datos que mejor se ajusta es la de una lista de listas binárias, dónde los valores binários representen la asistencia de un actor a una toma en particular. Esta estructura, entendida cómo una matriz "$N x M$", dónde "$N$" es el total de tomas y "$M$" el total de actores, nos indica (con 1: sí, y 0: no) si el actor "$n$" ha de asistir a la toma "$m$", en la posición "$(n, m)$" de la matriz. Ésta es justo la estructura que se nos brinda en el archivo de datos .xlsx

- Análisis posterior, con referencia al estudio de Alberto [1]:

La estructura que se ajusta mejor, es una lista de listas ````sol````, dónde cada lista contiene las tomas que se van a grabar en un determinado día. Con un 1 o un 0 en la posición de la lista correspondiente a cada toma. Además, con esta estructura podemos controlar fácilmente el número de días que se requieren para grabar todas las tomas con:

```python
sol: list[list[bool]]
dias = len(sol)
```

Además, podemos convertir la solución en un array de numpy, y comprobar que la cantidad de tomas por actor al día no supera el máximo establecido ````max_shots = 6````, con operaciones matriciales ````@````:

```python
sol: numpy.array
invalid_sol = any([any((sol[day] @ data) > max_shots) for day in range(len(sol))])
```

Dónde ````data```` es nuestro dataframe de actor por toma

__Según el modelo para el espacio de soluciones<br>__
__(*)¿Cual es la función objetivo?__

La función objetivo ````days()```` consiste en contar los días necesarios para grabar todas las tomas.

In [6]:
def days(sol):
    return len(sol)

In [7]:
# sin tener en cuenta si la solución es válida o no
days([[1,0,0],[0,1,1]])

2

Además, crearemos una función ````is_valid()```` para comprobar si una solución cumple las restricciones.

In [8]:
def is_valid(sol: np.array, data: np.array, max_shots: int = 6):
    
    # comprobar que se graben todas las tomas una vez
    if any(np.sum(sol, axis=0) != 1): return False  

    # comprobar que en un día no se requiera a un actor más de {max_shots} veces
    if any([any((sol[day] @ data) > max_shots) for day in range(len(sol))]): return False
    
    return True

In [9]:
# ejemplo: todas las tomas el mismo día, con un máximo de 2 tomas al día por actor
is_valid(
    sol = np.array([[1,1,1], [0,0,0]]),  # todas las tomas el mismo día
    data = np.array([[1,1,0],
                    [1,1,0],
                    [1,0,1]]),
    max_shots = 2)

False

__(*)¿Es un problema de maximización o minimización?__

Es un problema de minimización, ya que se pretende minimizar el número de días necesarios para grabar todas las tomas. Aunque por otro lado, se pretende maximizar el número de tomas por día, todo depende de cómo se plantee el espacio de soluciones y la variable a optimizar.

__Diseña un algoritmo para resolver el problema por fuerza bruta__

In [10]:
def brute_force_solve(data, max_shots=6):

    # Obtener el número mínimo de días teóricos
    min_day = (max(np.sum(data, axis = 0)) // max_shots) + 1 * (0 != max(np.sum(data, axis = 0)) % max_shots)

    # Preparar la estructura de datos solución
    solution = np.zeros((min_day, len(data)), dtype=int)
    solution[0] = 1

    days, shots = solution.shape
    c = 0  # contador de posición

    # Recorrer todas las posibles soluciones hasta encontrar la buena
    for pos in range(shots):
        c += 1
        for shot in range(c):
            
            for day in range(days):

                print(pos, shot, day)
                #print(solution)
                
                if is_valid(solution, data, max_shots):
                    
                    return solution
                    
                solution[day, shot] = 0
                solution[(day+1)%days, shot] = 1
        
    

In [11]:
a = np.array([[0, 1],
              [0, 1],
              [0, 1],
              [0, 1]])

In [12]:
sol = brute_force_solve(a, max_shots=2)

0 0 0
0 0 1
1 0 0
1 0 1
1 1 0
1 1 1
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
3 0 0
3 0 1
3 1 0
3 1 1
3 2 0
3 2 1
3 3 0
3 3 1


In [13]:
print(sol)

None


No he conseguido implementar bien el algoritmo para que dé soluciones válidas, aunque analizaré su complejidad actual ya que sería parecida a la de un algoritmo de fuerza bruta.

__Calcula la complejidad del algoritmo por fuerza bruta__

La complejidad del algoritmo por fuerza bruta es del orden de $O(dn^2)$ dónde $d$ son los mínimos días posibles de la solución, que a su vez se puede expresar cómo $n/m$ en el peor de los casos, dónde $m$ son el máximo de tomas por actor al día.

Así que se puede expresar también cómo $O(n^3/m)$

__(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta__

### 1. Solución por búsqueda aleatoria

Vamos a generar un algoritmo de generación de soluciones aleatorias, hasta encontrar una solución que nos dé un número de días suficientemente corto.

Aunque vamos a generar soluciones aleatorias, debemos tener en cuenta que existe un número máximo de días (solución menos óptima), al igual que un número mínimo (solución ideal de existencia no asegurada):
- **Número máximo de días:** El total de tomas dividido por el máximo de tomas por actor al día.

In [14]:
max_days = 1 + (len(data) // max_shots)
max_days

6

- **Número mínimo de días:** El número de tomas del actor con más participación dividido por el máximo de tomas por actor al día.

In [15]:
def min_days(data, max_shots):
    return (max(np.sum(data, axis = 0)) // max_shots) + 1 * (0 != max(np.sum(data, axis = 0)) % max_shots)

min_days(data, 6)

4

Dónde ````data```` es nuestro dataframe de actor por toma

In [16]:
def generate_random_solution(data: pd.DataFrame, days_objective: int, max_shots: int = 6):
    
    random_solution = np.zeros((days_objective, len(data)), dtype=int)  # Inicializar el array soluicón, dónde las filas representan los días, y las columnas las tomas
    assigned_days = np.random.randint(0, days_objective, size=len(data))  # Asignar cada toma a un día
    
    for day, toma in enumerate(assigned_days):
        random_solution[toma, day] = 1  # Transformar la asignación de días a formato booleano

    return random_solution

Ahora, vamos a diseñar un algoritmo que recorra un bucle hasta encontrar una solución válida. Además, incluiremos un control para tiempos de iteración demasiado largos, que podremos acotar con el parámetro ````max_iter````.

In [17]:
def solve_for_N_days(ndays: int, data, max_iter=1000):
    for _ in range(max_iter):
        proposed_sol = generate_random_solution(data, days_objective = ndays)
        if is_valid(proposed_sol, data):
            return proposed_sol

    return None

Y, en caso de no encontrar una solución en ese número de iteraciones, buscaremos una solución con un día más.

In [18]:
def random_solve(data: pd.DataFrame, max_shots=6, max_iter=1000):

    # Obtener el número máximo y mínimo de días teóricos
    min_day = min_days(data, max_shots)
    max_days = 1 + (len(data) // max_shots)
    
    # inicialización de la variable de control de días
    best_sol_days = 9999999

    # loop de búsqueda de soluciones
    for ndays in range(min_day, max_days + 1):
        solution = solve_for_N_days(ndays, data, max_iter)
        if solution is not None:
            return ndays, solution 

In [19]:
%%time
best_sol_days, best_sol_found = random_solve(data, max_iter=1000)
best_sol_days, is_valid(best_sol_found, data), best_sol_found

CPU times: total: 31.2 ms
Wall time: 21.9 ms


(4,
 True,
 array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
         1, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 1]]))

Se puede observar cómo el algoritmo random_solve consigue encontrar una solución para grabar todas las tomas en 4 días. De hecho, aproximadamente entre un 6-7% de las soluciones generadas por la función ````generate_random_solution()```` serán válidas para 4 días:

In [20]:
def percentage_of_good_solutions(ndays, data):
    print(f"Porcentaje de soluciones válidas generadas para {ndays} días:", sum([is_valid(generate_random_solution(data, days_objective=ndays), data) for _ in range(10000)]) / 100, "%")

In [21]:
for i in range(1, max_days + 1):
    percentage_of_good_solutions(i, data)

Porcentaje de soluciones válidas generadas para 1 días: 0.0 %
Porcentaje de soluciones válidas generadas para 2 días: 0.0 %
Porcentaje de soluciones válidas generadas para 3 días: 0.0 %
Porcentaje de soluciones válidas generadas para 4 días: 6.51 %
Porcentaje de soluciones válidas generadas para 5 días: 37.68 %
Porcentaje de soluciones válidas generadas para 6 días: 63.04 %


Observamos que efectivamente no se encuentran soluciones para menos de 4 días, y a medida que se incrementan los días, la probabilidad de encontrar una solución es mayor. Con lo que se justifica el hecho de buscar soluciones en un número mayor de días si no se encuentran soluciones para un determinado número de días objetivo.

### 2. Solución por optimización de espacio (incompleta)

In [22]:
def unavailable_slots(solution, data, max_shots) -> np.array(bool) :
    return (solution @ data == max_shots)

In [23]:
unavailable_slots(best_sol_found, data, max_shots)

1      2      3      4      5      6      7      8      9     10
0  False  False  False  False  False  False  False  False  False  False
1  False  False  False  False  False  False  False  False  False  False
2   True  False  False  False  False  False  False  False  False  False
3   True  False  False  False  False  False  False  False  False  False

In [24]:
def score_by_day(shot, solution, data, max_shots) -> np.array(int):
    '''
    Returns the fitting score of a shot in a given day.

    The fitting score is the measure of how well a shot would fit in a day.
    Measured with a logical XOR operation between the available actor slots
    (from func unavailable_slots()) and the required actors in "shot".
    '''
    return np.sum(np.logical_xor(shot, unavailable_slots(solution, data, max_shots)), axis=1)

In [25]:
def recursive_solve(solution, data, max_shots):
    is_pending = np.logical_not(np.sum(solution, axis=0))

    if any(is_pending):
        pending_shots = data[:][is_pending]  # Get only the shots that are still not in the solution
        scores = np.array([score_by_day(np.array(shot), solution, data, max_shots) for _, shot in
                           pending_shots.iterrows()]).T  # Compute their xor scores
        day, i = np.unravel_index(np.argmax(scores), scores.shape)  # Get the day and shot that best fit together
        shot_id = pending_shots.index[i] - 1
        solution[day, shot_id] = 1

        recursive_solve(solution, data, max_shots)

    return solution

In [26]:
def solve(data, max_shots):

    min_day = min_days(data, max_shots)
    solution = np.zeros((min_day, len(data)), dtype=int)
    recursive_solve(solution, data, max_shots)
    return solution

In [27]:
min_day = min_days(data, max_shots)
solution = np.zeros((min_day, len(data)), dtype=int)
is_pending = np.logical_not(np.sum(solution, axis=0))
np.sum(is_pending)

30

In [28]:
sol = solve(data, 6)
sol

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]])

In [29]:
is_valid(sol, data)

False

No he podido conseguir implementar este algoritmo para que diera soluciones válidas

__(*)Calcula la complejidad del algoritmo__

Análisis de complejidad de los algoritmos:

### 1. Por solución aleatoria

- ````generate_random_solution()````: $O(n)$ La complejidad depende del número de tomas de forma lineal.

- ````is_valid()````: $O(1)$ Depende del número de días de la solución, que acostumbran a ser pocos.

- ````solve_for_N_days()````: $O(n \cdot$ ````max_iter```` $)$ Depende linealmente del número de iteraciones máximas que indiquemos o menor, que es cuando se encuentra una solución.

- ````random_solve()````: $O(n \cdot$ ````min_days```` $ \cdot $ ````max_iter```` $)$ Depende de los datos específicos del problema, sobretodo de la densidad de población. La densidad de población es la cantidad de unos del dataset dividido por el total de elementos.

En este caso tiene más sentido hablar de probabilidiad de encontrar una solución válida, que de complejidad. Por eso el algoritmo no puede encontrar soluciones para una densidad del dataset un poco mayor, ya que la probabilidad de encontrar una solución se reduce con la falta de espacio.

### 2. Por optimización de espacio (incompleto)

- ````unavailable_slots()````: $O(1)$ Ya que se computa con operaciones matriciales optimizadas por numpy

- ````score_by_day()````: $O(1)$ Ya que usa también operaciones matriciales optimizadas de numpy

- ````recursive_solve()````: $O(n^2)$ Depende principalmente del número de tomas al cuadrado, ya que a medida que se van asignando espacios, la función se vuelve a llamar a sí misma con menos tomas cada vez. 

- ````solve()````:  El mismo orden que la anterior función, ya que simplemente sirve de encapsulamiento.

__Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios__

Debemos asegurar que todas las tomas tienen al menos un actor presente. Podemos aprovechar la lógica de la función ````generate_random_solution()````:

In [30]:
def generate_random_dataset(shots: int, actors: int, min_population_fraction: float):

    if (min_population_fraction <= 0) or (min_population_fraction > 1):
        raise ValueError("min_population_fraction should be a value in the range (0, 1]")
    
    random_dataset = np.zeros((shots, actors), dtype=int)  # Inicializar el array dataset, dónde las filas representan los tomas, y las columnas los actores
    
    while (np.sum(random_dataset) / random_dataset.size) < min_population_fraction:  # Asegurar el mínimo de población
        assigned_actors_to_shot = np.random.randint(0, actors, size=shots)  # Asignar cada toma a un actor
        for shot, actor in enumerate(assigned_actors_to_shot):
            random_dataset[shot, actor] = 1

    return random_dataset

In [31]:
random_dataset = generate_random_dataset(shots=30, actors=22, min_population_fraction=.4)

In [32]:
random_dataset

array([[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 1,

In [33]:
if all(np.sum(random_dataset, axis=1) > 0):
    print("Todas las tomas tienen al menos un actor")
else:
    print("Hay tomas sin actores")

Todas las tomas tienen al menos un actor


__Aplica el algoritmo al juego de datos generado__

In [34]:
max_shots = 6
max_iter = 10000000

In [35]:
min_day = (max(np.sum(random_dataset, axis = 0)) // max_shots) + 1 * (0 != max(np.sum(random_dataset, axis = 0)) % max_shots)
min_day

3

In [36]:
%%time
best_sol_days, best_sol_found = random_solve(random_dataset, max_shots, max_iter)
best_sol_days, is_valid(best_sol_found, random_dataset), best_sol_found

CPU times: total: 46.9 ms
Wall time: 40.9 ms


(3,
 True,
 array([[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
         0, 1, 0, 0, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
         1, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 1, 1, 1, 0, 0, 0]]))

__Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo__

[1] Alberto Caldas Lima, [Aplicación de algoritmos heurísticos para optimizar el coste de doblaje de películas](http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_759.pdf)

[2] Wikipedia, [Bell Number](https://en.wikipedia.org/wiki/Bell_number)

[3] Rajeev Agrawal, [Bell Numbers (Number of ways to Partition a Set)](https://www.geeksforgeeks.org/bell-numbers-number-of-ways-to-partition-a-set/)

__Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño__

Respuesta

La verdad es que por falta de tiempo en mi vida personal me da pena no poder haber llegado hasta el fondo del algoritmo basado en operaciones XOR. Me gustaría seguir trabajando en ello, ya que el razonamiento detrás del algoritmo tiene cierta lógica.

Aún así, creo que el siguiente paso en el estudio del problema sería parar a buscar alternativas, cómo se explican en el paper de Casas Lima [1], y que seguramente nos den una solución elegante y de baja complejidad para el problema. También sería interesante hacer un modelado matemáticamente riguroso del problema, para entender mejor las limitaciones y algoritmos aplicables a partir de la teoría.